In [13]:

%load_ext sql
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%%sql
CREATE TABLE IF NOT EXISTS title_basics (
    tconst VARCHAR(25) NOT NULL,
    titleType TEXT,
    primaryTitle TEXT,
    originalTitle TEXT,
    isAdult BOOLEAN,
    startYear INT,
    endYear INT,
    runtimeMinutes INT,
    genres TEXT
);


Running query in 'duckdb:///:memory:'

Count


In [15]:
%%sql
CREATE TABLE IF NOT EXISTS name_basics (
    nconst VARCHAR(25) NOT NULL,
    primaryName TEXT,
    birthYear INT,
    deathYear INT,
    primaryProfession TEXT,
    knownForTitles TEXT
);


Running query in 'duckdb:///:memory:'

Count


In [16]:
%%sql
CREATE TABLE IF NOT EXISTS title_episode (
    tconst VARCHAR(15),
    parentTconst VARCHAR(15),
    seasonNumber INT,
    episodeNumber INT
);


Running query in 'duckdb:///:memory:'

Count


In [17]:
%%sql
CREATE TABLE IF NOT EXISTS name_basics (
    nconst VARCHAR(25) NOT NULL,
    primaryName TEXT,
    birthYear INT,
    deathYear INT,
    primaryProfession TEXT,
    knownForTitles TEXT
);


Running query in 'duckdb:///:memory:'

Count


In [18]:
%%sql
CREATE TABLE IF NOT EXISTS title_episode (
    tconst VARCHAR(15),
    parentTconst VARCHAR(15),
    seasonNumber INT,
    episodeNumber INT
);


Running query in 'duckdb:///:memory:'

Count


In [19]:
%%sql
CREATE TABLE IF NOT EXISTS title_crew (
    tconst VARCHAR(15),
    directors TEXT,
    writers TEXT
);


Running query in 'duckdb:///:memory:'

Count


In [20]:
%%sql
CREATE TABLE IF NOT EXISTS title_ratings (
    tconst VARCHAR(15),
    averageRating FLOAT,
    numVotes INT
);


Running query in 'duckdb:///:memory:'

Count


In [21]:
%%sql
CREATE TABLE IF NOT EXISTS title_akas (
    titleId VARCHAR(15),
    ordering INT,
    title TEXT,
    region TEXT,
    language TEXT,
    types VARCHAR(50),
    attributes VARCHAR(255),
    isOriginalTitle BOOLEAN
);


Running query in 'duckdb:///:memory:'

Count


In [22]:
%%sql
CREATE TABLE IF NOT EXISTS title_principals (
    tconst VARCHAR(15),
    ordering INT,
    nconst VARCHAR(15),
    category TEXT,
    job TEXT,
    characters TEXT
);


Running query in 'duckdb:///:memory:'

Count


In [23]:
# Cell 9
import pandas as pd
import requests
import io
import duckdb

# Reconnect to DuckDB (in-memory database)
con = duckdb.connect(database=':memory:', read_only=False)

# URLs and corresponding table names
gz_file_urls = {
    "https://datasets.imdbws.com/title.ratings.tsv.gz": "title_ratings",
    "https://datasets.imdbws.com/title.principals.tsv.gz": "title_principals",
    "https://datasets.imdbws.com/title.episode.tsv.gz": "title_episode",
    "https://datasets.imdbws.com/title.crew.tsv.gz": "title_crew",
    "https://datasets.imdbws.com/title.basics.tsv.gz": "title_basics",
    "https://datasets.imdbws.com/title.akas.tsv.gz": "title_akas",
    "https://datasets.imdbws.com/name.basics.tsv.gz": "name_basics"
}

for url, table_name in gz_file_urls.items():
    print(f"Processing {table_name}")
    response = requests.get(url)
    df = pd.read_csv(io.BytesIO(response.content), delimiter='\t', compression='gzip', low_memory=False)

    # Register DataFrame as a view in DuckDB
    con.register(table_name+'_view', df)

    # Insert data into DuckDB table
    con.execute(f"INSERT INTO {table_name} SELECT * FROM {table_name+'_view'}")


Processing title_ratings


CatalogException: Catalog Error: Table with name title_ratings does not exist!
Did you mean "title_ratings_view"?

In [24]:
%sql SELECT * FROM information_schema.tables WHERE table_schema = 'main';


Running query in 'duckdb:///:memory:'

table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
memory,main,title_basics,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,name_basics,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,title_episode,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,title_crew,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,title_ratings,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,title_akas,BASE TABLE,None,None,None,None,None,YES,NO,None
memory,main,title_principals,BASE TABLE,None,None,None,None,None,YES,NO,None
